In [1]:
import os
from openai import AzureOpenAI

In [4]:
def get_chat_answer(prompt, context, chat_model = "RAG-gpt-35"):
    '''
    This function returns the response from the RAG model and the GPT-3.5 model
    
    Args:
    prompt: str
    context: list
    chat_model: str

    Returns:
    rag_answer: str
    gpt_answer: str

    '''
    
    client = AzureOpenAI(
        api_key=os.getenv("64e7c187bf824fb6a1ed25d889592335"),  
        api_version=os.getenv("2023-12-01-preview"),
        azure_endpoint=os.getenv("https://ragembedding.openai.azure.com/")
    )
    RAG_context = "Here are chunks similar to the prompt from extra backstory of League of Legends. You must use them to guide your response as long as they align with the prompt in some degree: \n In the game League of Legends, "
    for i in range(len(context)):
        RAG_context += f" {context[i]}"
    # print(RAG_context)

    messages = [
    {"role": "system", "content": "You are a person who are very familar League of Legends, and you are good at summry from the chunks."},
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": RAG_context}
  ]
    rag_answer =  client.chat.completions.create(
        model = chat_model,
        messages = messages,
        stream = True
    )

    messages = [
    {"role": "system", "content": "You are a gaming encyclopedia."},
    {"role": "user", "content": prompt}
    ]

    gpt_answer =  client.chat.completions.create(
        model = chat_model,
        messages = messages,
        stream = True
    )

    return rag_answer, gpt_answer

In [13]:
from scripts.retrieval import topk_retrieval, get_chunk_ids
from scripts.data_storage import get_index
database = get_index()
champion_list = ['Ahri',
'Akali',
'Wukong',
'Teemo',
'Volibear',
'Yasuo',
'Lux',
'Zoe',
'Jhin',
'Leesin',
'Leblanc',
'Caitlyn'
]




UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'X-Cloud-Trace-Context': '5948af1b6e18298202e85b43bf0a5ce1', 'Date': 'Fri, 26 Apr 2024 07:26:29 GMT', 'Content-Type': 'text/html', 'Server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


In [5]:
RAG_answer=[]
No_RAG_answer=[]
for champion in champion_list:
    prompt = f"Write a comic plot about {champion} in league of legend."
    _, context = topk_retrieval(
            ids=get_chunk_ids(), query=prompt, index=database, k=5
        )
    rag, gpt = get_chat_answer(
                prompt, context, chat_model='RAG-gpt-35')
    RAG_answer.append(rag)
    No_RAG_answer.append(gpt)

NameError: name 'champion_list' is not defined

In [ ]:
client = AzureOpenAI(
        api_key=os.getenv("AZURE_OPENAI_KEY"),  
        api_version=os.getenv("OPENAI_API_VERSION"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
    )


In [ ]:
results = []
for rag, gpt in zip(RAG_answer,No_RAG_answer):
    response = client.chat.completions.create(

    messages=[
        {"role": "system", "content": "You are a helpful League of Legends comic plots critic."},
        {"role": "user", "content": "Which of the following 2 comic plot is most close to League of Legend's style? Give me the index, like the first/second."
        +"1. "+rag+'\n'
        +"2. "+gpt

        }
    ]
    )
    print(response.choices[0].message.content)
    results.append(response.choices[0].message.content)